In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import glob
import joblib

# Load dataset class (same as before)
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors if preprocessors else []
    
    def load(self, image_paths, verbose=1):
        data, labels = [], []
        class_names = sorted(os.listdir('Helmet_Dataset'))
        
        for class_idx, class_name in enumerate(class_names):
            class_path = os.path.join('Helmet_Dataset', class_name)
            image_paths = glob.glob(os.path.join(class_path, "*.jpg")) + \
                         glob.glob(os.path.join(class_path, "*.png")) + \
                         glob.glob(os.path.join(class_path, "*.jpeg"))
            
            for img_path in image_paths:
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, (224, 224))  # MobileNetV2 input size
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    data.append(image)
                    labels.append(class_idx)
        
        return np.array(data), np.array(labels), class_names

print("Loading dataset...")
dl = SimpleDatasetLoader()
(data, labels, class_names) = dl.load('Helmet_Dataset')
print(f"Loaded {len(data)} images: {class_names}")

# Split data
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42, stratify=labels)

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    fill_mode='nearest'
)

# Build lightweight MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model for transfer learning

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Training MobileNetV2 model...")
history = model.fit(
    datagen.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX)//32,
    epochs=15,
    validation_data=(testX, testY),
    verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(testX, testY)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

# Save model
model.save('Models/helmet_mobilenetv2.h5')
print("Model saved as 'helmet_mobilenetv2.h5'")


C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Plea

Loading dataset...
Loaded 671 images: ['Helmet', 'Person_no_helmet', 'no_person']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 21s 2us/step
Training MobileNetV2 model...
Epoch 1/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4628 - loss: 1.1508 - val_accuracy: 0.6429 - val_loss: 0.6993
Epoch 2/15
 1/15 ━━━━━━━━━━━━━━━━━━━━ 8s 572ms/step - accuracy: 0.6250 - loss: 0.8134

C:\Users\Lenovo\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 223ms/step - accuracy: 0.6250 - loss: 0.8134 - val_accuracy: 0.6131 - val_loss: 0.7043
Epoch 3/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.6348 - loss: 0.7705 - val_accuracy: 0.5952 - val_loss: 0.7086
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 218ms/step - accuracy: 0.8125 - loss: 0.6737 - val_accuracy: 0.6488 - val_loss: 0.6891
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.6454 - loss: 0.7113 - val_accuracy: 0.7560 - val_loss: 0.5767
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 218ms/step - accuracy: 0.7188 - loss: 0.6657 - val_accuracy: 0.7619 - val_loss: 0.5649
Epoch 7/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.7282 - loss: 0.6211 - val_accuracy: 0.7500 - val_loss: 0.5592
Epoch 8/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 222ms/step - accuracy: 0.7812 - loss: 0.5890 - val_accuracy: 0.7500 - val_loss: 0.5454
Epoch 9/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.7219 - loss: 0.6238 - val_accuracy: 0.7976 - val_loss: 0.


Final Test Accuracy: 0.7976
Model saved as 'helmet_mobilenetv2.h5'


In [ ]:
import tensorflow as tf
import cv2
import numpy as np

# Load model
model = tf.keras.models.load_model('Models/helmet_mobilenetv2.h5')
class_names = ['Helmet', 'no_person', 'Person_no_helmet']

def predict_single_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize
    
    predictions = model.predict(img)[0]
    predicted_class = np.argmax(predictions)
    confidence = predictions[predicted_class]
    
    print(f"Predicted: {class_names[predicted_class]} (Confidence: {confidence:.2%})")
    return class_names[predicted_class], confidence

# Usage
result, conf = predict_single_image('Test_Files/Person_with_helmet_2.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: Person_no_helmet (Confidence: 100.00%)
